In [2]:
import duckdb
import os
from dotenv import load_dotenv
import duckdb

load_dotenv()  # loads .env into environment

db_path = os.getenv("DUCKDB_PATH", "data.duckdb")  # default fallback

con = duckdb.connect(db_path)  # this creates/opens the file

# Create a table if it doesn't exist

# con.execute(
#     """
# CREATE TABLE IF NOT EXISTS platform_metrics (
#     car_id       TEXT,
#     platform     TEXT,
#     run_ts       TIMESTAMP,
#     metrics      JSON,
#     PRIMARY KEY (car_id, platform, run_ts)
# );

# CREATE TABLE IF NOT EXISTS overall_cache (
#     car_id       TEXT,
#     run_ts       TIMESTAMP,
#     metrics      JSON,
#     PRIMARY KEY (car_id, run_ts)
# );
# """
# )

In [7]:
from datetime import datetime
from pydantic import BaseModel
import json

# Sample Pydantic model


class Metrics(BaseModel):
    num_comments: int
    avg_sentiment_score: float
    most_common_sentiment: str
    likes: int
    shares: int
    plays: int
    collections: int
    engagement_score: float
    overall_sentiment_score: float


m = Metrics(
    num_comments=42,
    avg_sentiment_score=0.75,
    most_common_sentiment="positive",
    likes=100,
    shares=8,
    plays=500,
    collections=5,
    engagement_score=0.85,
    overall_sentiment_score=0.78,
)

con.execute(
    """
    INSERT INTO platform_metrics (car_id, platform, run_ts, metrics)
    VALUES (?, ?, ?, ?)
""",
    ["car123", "tiktok", datetime.now(), m.model_dump_json()],
)

In [4]:
import json
rows = con.execute(
    """
    SELECT car_id, metrics
    FROM (
      SELECT *, row_number() OVER (PARTITION BY car_id ORDER BY run_ts DESC) AS rn
      FROM platform_metrics
      WHERE platform = 'instagram'
    )
    WHERE rn = 1
    """
).fetchall()

for car_id, metrics_blob in rows:
    metrics = json.loads(metrics_blob)
    print(car_id, metrics)

ferrari 355 gtb {'num_comments': 3, 'avg_sentiment_score': 0.38641161719957984, 'most_common_sentiment': 'neutral', 'likes': 133, 'shares': 0, 'plays': 0, 'collections': 0, 'engagement_score': 14050.0, 'overall_sentiment_score': 0.38675669246982314, 'topics': None, 'topic_sentiments': None, 'price_series': None, 'popularity_series': None}
ferrari dino 246 {'num_comments': 9, 'avg_sentiment_score': 0.39505031875201635, 'most_common_sentiment': 'neutral', 'likes': 589, 'shares': 0, 'plays': 0, 'collections': 0, 'engagement_score': 59900.0, 'overall_sentiment_score': 0.4028284772874316, 'topics': None, 'topic_sentiments': None, 'price_series': None, 'popularity_series': None}
lamborghini 350 gt {'num_comments': 13, 'avg_sentiment_score': 0.4059978370865186, 'most_common_sentiment': 'neutral', 'likes': 1019, 'shares': 0, 'plays': 0, 'collections': 0, 'engagement_score': 103400.0, 'overall_sentiment_score': 0.40140688966202515, 'topics': None, 'topic_sentiments': None, 'price_series': None,